In [1]:
from alignment import *
from parsing import parse_matrix_and_gap_cost_in_subst_matrix,read_n_fasta
from msa_sp_score_3k import compute_sp_score
import os

In [2]:
matrix_path = 'input/subst_matrix.txt'
substitution_matrix = parse_matrix_and_gap_cost_in_subst_matrix(matrix_path)

Read and align the sequences from 1-3 to 1-6.

In [3]:
list_of_seq = read_n_fasta('experiments/brca1-testseqs.fasta',6)
results = []
for i in [3,4,5,6]:
        M = two_approx_algorithm_for_MSA(list_of_seq[:i], substitution_matrix)  ########## algorithm TO REPLACE
        alignment_approx_outputfile = 'presentation/aligned_' + str(i)
        approx_alignment = print_alignment(M,alignment_approx_outputfile) ## print alignment to file
    

Compute scores from alignments:

In [4]:
for i in [3,4,5,6]:
    ####### approx
    testfile = 'presentation/aligned_' + str(i)
    if os.path.isfile(testfile):   
        score = compute_sp_score(testfile)
        print("The optimal score is in seqs 1 -",i,":",score)

The optimal score is in seqs 1 - 3 : 792
The optimal score is in seqs 1 - 4 : 2345
The optimal score is in seqs 1 - 5 : 3295
The optimal score is in seqs 1 - 6 : 6005


In [7]:
seq_full = read_n_fasta('experiments/brca1-full.fasta')
len(seq_full)

8

In a sequence, there are nucleotides that are not A,C,G or T

In [11]:
for seq in seq_full:
    print(set(seq))

{'A', 'C', 'G', 'T'}
{'A', 'C', 'G', 'T'}
{'G', 'T', 'A', 'C'}
{'G', 'A', 'C', 'T'}
{'A', 'C', 'G', 'T'}
{'G', 'A', 'C', 'T'}
{'A', 'C', 'G', 'T'}
{'T', 'G', 'C', 'N', 'R', 'S', 'A'}


In [13]:
import random
# Replace 'N', 'R' or 'S' with a random nucleotide
clean_seq_full = []
for sequence in seq_full:
    replaced_sequence = ''
    for nucleotide in sequence:
        if nucleotide == 'N' or nucleotide == 'R' or nucleotide == 'S' :
            replaced_sequence += random.choice(['A', 'C', 'T', 'G'])
        else:
            replaced_sequence += nucleotide
    clean_seq_full.append(replaced_sequence)

for seq in clean_seq_full:
    print(set(seq))

{'A', 'C', 'G', 'T'}
{'A', 'C', 'G', 'T'}
{'G', 'T', 'A', 'C'}
{'G', 'A', 'C', 'T'}
{'A', 'C', 'G', 'T'}
{'G', 'A', 'C', 'T'}
{'A', 'C', 'G', 'T'}
{'G', 'T', 'A', 'C'}


Align the 8 sequences:

In [14]:
M = two_approx_algorithm_for_MSA(clean_seq_full, substitution_matrix) ########## algorithm TO REPLACE
alignment_outputfile = 'presentation/full_aligned'
approx_alignment = print_alignment(M,alignment_outputfile) ## print alignment to file

TypeError: 'NoneType' object is not subscriptable

In [ ]:
compute_sp_score(alignment_outputfile)